# ResNetを使った画像認識を体験してみよう！

**torchライブラリのインポート**  
※Pytorchはディープラーニングを簡単に使用するためのフレームワークであり、torchはPytorchの中のライブラリの一つ

In [ ]:
import torch
print(torch.__version__)

2.1.0+cu121


In [ ]:
torch.cuda.is_available()

True

**訓練済みモデルの表示**    
* 大文字で表示されたものは利用可能なモデルのPythonクラス
* 小文字で表示されたものは大文字で表示されたクラスのインスタンス化されたモデルを返す関数や事前訓練済みのモデルをロードするためのメソッド

In [ ]:
from torchvision import models
dir(models)

**AlexNetオブジェクトの呼び出し**  
※この時点ではパラメータ変数が訓練されていない初期状態

In [ ]:
alexnet = models.AlexNet()

**訓練済みのResNetオブジェクトの呼び出し**  
※ここでは101層の畳み込みニューラルネットワークを使用

In [ ]:
resnet = models.resnet101(pretrained=True)

**ResNet101の中身を確認**  
ここでは一行が一つのモジュール(Pytorchにおける)を表している  
* Pythonのモジュールはコードの組織化や再利用のためのファイル単位の概念  
* PyTorchのモジュールはニューラルネットワークの構成要素を定義するためのクラスベースの概念

In [ ]:
resnet

**入力画像の前処理関数を定義**


1.   256 × 256のサイズに変換
2.   画像の中央を224 × 224のサイズで切り取り
3.   tensorに変換(色、高さ、幅の３次元配列)
4.   設定した平均値と標準偏差で標準化

**正規化と標準化**  
お互い特徴量の単位が違っていたりしたときに、各次元の関係をわかりやすくするために使われる。ニューラルネットワークの学習が安定する。画像処理においては、照明の違いや露出の違いなどによる影響を減少させることができる。過学習を防ぎ、汎化性能を向上させることに役立つ
* 正規化: データの最小値を0、最大値を1の範囲に収める
    * 外れ値の影響を受けやすい
    * 最大値と最小値がわかっている時に使われる

* 標準化: 平均を0, 標準偏差を1とする

※transforms.Compose は、PyTorchの torchvision ライブラリの一部であり、主に画像処理のための複数のトランスフォーム（変換）を組み合わせるために使用されます。これは、機械学習モデルのデータ前処理やデータ拡張のステップに非常に役立ちます。

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
# print(type(preprocess))

<class 'torchvision.transforms.transforms.Compose'>


**マウント**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**テスト画像を読み込み・表示**

In [ ]:
from PIL import Image
import numpy as np

img = Image.open('/content/drive/MyDrive/Colab Notebooks/bobby.jpg')
# n = np.array(img)
# print(n)
# print(n.shape)
img

**読み込んだ画像を用意しておいた前処理用の関数に通す**

In [ ]:
img_t = preprocess(img)
print(img_t)
print(type(img_t))

**第1次元に次元を追加**

In [ ]:
batch_t = torch.unsqueeze(img_t, 0)

**推論を行うためにネットワークをevalに変更**

In [ ]:
resnet.eval()

**ImageNetの各クラスに対応するスコアベクトルを計算**

In [ ]:
out = resnet(batch_t)
print(out)
print(type(out))
print(out.shape)

**ラベルをリストアップしたファイルを読み込み**

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/imagenet_classes.txt') as f:
  labels = [label.strip() for label in f.readlines()]
print(labels)

ラベルごとのスコアの最大値とそのインデックスを取得

In [ ]:
val, index = torch.max(out, 1)
print(val)
print(index)

**最もスコアの高かったラベルとその予測に対する信頼度を表示**

In [ ]:
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()

('golden retriever', 96.57185363769531)

**スコアの高かったラベルとその予測に対する信頼度を上から５つ表示**

In [ ]:
_, indices = torch.sort(out, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('golden retriever', 96.57185363769531),
 ('Labrador retriever', 2.6082630157470703),
 ('cocker spaniel, English cocker spaniel, cocker', 0.26996269822120667),
 ('redbone', 0.17958903312683105),
 ('tennis ball', 0.10991978645324707)]